In [1]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure, output_file, show
from bokeh.layouts import row,column
from bokeh.models import ColumnDataSource, PanTool,ResetTool,BoxZoomTool,WheelZoomTool,UndoTool,HoverTool
from bokeh.palettes import Spectral

from Project import dataCollection
from src.algorithm import EMPTY_BLOCK_MOVEMENT
from src.runner import paralleledRunner

In [2]:
# Not returning anything, but output a file
def plotting(df:pd.DataFrame) -> None:
    # Define tools needed
    tools = [PanTool(),ResetTool(),BoxZoomTool(),WheelZoomTool(dimensions='width'),WheelZoomTool(dimensions='height'),UndoTool()]
    
    #Define colors
    colors = Spectral[11]
    
    # Define datascource
    base, lower, upper = [], [], []
    datasource_dict = {'x': df.C.unique()}
    for num_of_y in range(0,len(df.MN.unique())):
        datasource_dict['y'+str(num_of_y)] = list(df.loc[df.MN == df.MN.unique()[num_of_y],'Steps'].apply(lambda x: np.array(x).mean()))
        datasource_dict['y'+str(num_of_y+len(df.MN.unique()))] = list(df.loc[df.MN == df.MN.unique()[num_of_y],'Density'])
    datasource = ColumnDataSource(datasource_dict)
    
    # First figure showing the relationship between C and Steps with given M,N combination, Second show C-Density
    p1 = figure(background_fill_color="white", plot_width=1200, plot_height=400, title = "C-Steps Plot",tools=tools)
    p2 = figure(background_fill_color="white", plot_width=1200, plot_height=400, title = "C-Density Plot",tools=tools)
    for num_of_y in range(0,len(df.MN.unique())):
        p1.line('x', 'y'+str(num_of_y), source=datasource, legend_label='(M,N) = '+str(df.MN.unique()[num_of_y]),muted_alpha=0.2,color=colors[num_of_y])
        p2.line('x', 'y'+str(num_of_y+len(df.MN.unique())), source=datasource, legend_label='(M,N) = '+str(df.MN.unique()[num_of_y]),muted_alpha=0.2,color=colors[num_of_y])
        for i in range(0,len(df.C.unique())):
            y1 = list(df.loc[df.MN == df.MN.unique()[num_of_y],'Steps'])[i]
            p1.circle(x=df.C.unique()[i], y=y1, color=colors[num_of_y],muted_alpha=0.2)
            
    p1.legend.click_policy="mute"
    p2.legend.click_policy="mute"
    
    #ouput file
    show(column(p1, p2))
    output_file("stats.html", title="stats")

In [3]:
df = dataCollection()

paralleledRunner(X=20,Y=20,C=1,Loop=100,block=1,fnc=<function EMPTY_BLOCK_MOVEMENT at 0x7fb79ece3430>,N=None,P=None) Starting
Using 0.06558704376220703 seconds
paralleledRunner(X=40,Y=20,C=1,Loop=100,block=1,fnc=<function EMPTY_BLOCK_MOVEMENT at 0x7fb79ece3430>,N=None,P=None) Starting
Using 0.05893588066101074 seconds
paralleledRunner(X=20,Y=40,C=1,Loop=100,block=1,fnc=<function EMPTY_BLOCK_MOVEMENT at 0x7fb79ece3430>,N=None,P=None) Starting
Using 0.056524038314819336 seconds
paralleledRunner(X=20,Y=20,C=1,Loop=100,block=5,fnc=<function EMPTY_BLOCK_MOVEMENT at 0x7fb79ece3430>,N=None,P=None) Starting
Using 0.06113147735595703 seconds
paralleledRunner(X=40,Y=20,C=1,Loop=100,block=5,fnc=<function EMPTY_BLOCK_MOVEMENT at 0x7fb79ece3430>,N=None,P=None) Starting
Using 0.05833745002746582 seconds
paralleledRunner(X=20,Y=40,C=1,Loop=100,block=5,fnc=<function EMPTY_BLOCK_MOVEMENT at 0x7fb79ece3430>,N=None,P=None) Starting
Using 0.057265281677246094 seconds
paralleledRunner(X=20,Y=20,C=1,Loop=10

In [4]:
df.to_csv('Output_noindex.csv', index = False)
df.to_csv('Output.csv')

In [ ]:
plotting(df)